# 3 - Coloring problem

#### Theorem (Planar graph 3-colorings, Penrose 1971)
The number K of proper 3- edge-colorings of a planar 3-regular graph is obtained by replacing each node with an order-3 **epsilon tensor**, replacing each edge with a wire, and then contracting the resulting tensor network.

> Roger Penrose, “Applications of negative dimensional tensors,” in Combinatorial Mathematics and its Applications, edited by D. Welsh (Academic Press, New York, 1971) pp. 221–244.

In [71]:
using OMEinsum, LinearAlgebra

In [72]:
# define the symmetrizer tensor
s_tensor(n::Int) = map(x->Int(length(unique(x.I)) == n), CartesianIndices(fill(n,n)|>Tuple))
I3 = Matrix(I, 3,3)
ϵ = s_tensor(3)

3×3×3 Array{Int64,3}:
[:, :, 1] =
 0  0  0
 0  0  1
 0  1  0

[:, :, 2] =
 0  0  1
 0  0  0
 1  0  0

[:, :, 3] =
 0  1  0
 1  0  0
 0  0  0

## A two-node 3-regular graph


<img src="images/twonode.png" width="200px"/>

In [73]:
# einstein summation notation
einsum(((1,2,3), (1,2,3)), (ϵ, ϵ), ())

0-dimensional Array{Int64,0}:
6

<img src="images/twonode_all.png" width="800px"/>

## Complete Graph $K_4$

In [11]:
einsum(((1,2,3), (3,4,5), (5,2,6), (6,4,1)), (ϵ, ϵ, ϵ, ϵ), ())

0-dimensional Array{Int64,0}:
6

## Peterson graph


<img src="images/peterson.png" width="400px"/>

In [69]:
edges = [(1,6), (2,7), (3,8), (4,9), (5,10), (1,2), (2,3), (3,4), (4,5), (5,1), (6,8), (8,10), (10,7), (7,9), (9,6)];

In [6]:
# compute the edges of the dual graph
function dual_graph(edges)
    vertices = [Int[] for i in 1:maximum(union(edges...))]
    for (i, edge) in enumerate(edges)
        for v in edge
            push!(vertices[v], i)
        end
    end
    return Tuple.(vertices)
end

dual_graph (generic function with 1 method)

In [7]:
dg = dual_graph(edges)

10-element Array{Tuple{Int64,Int64,Int64},1}:
 (1, 6, 10) 
 (2, 6, 7)  
 (3, 7, 8)  
 (4, 8, 9)  
 (5, 9, 10) 
 (1, 11, 15)
 (2, 13, 14)
 (3, 11, 12)
 (4, 14, 15)
 (5, 12, 13)

In [8]:
einsum(Tuple(dg), Tuple(ϵ for i=1:length(dg)), ())

0-dimensional Array{Int64,0}:
0

## $K_{3,3}$ Graph


<img src="images/k33.png" width="400px"/>

In [9]:
edges = [(1,2), (2,3), (3,4), (4,5), (5,6), (6,1), (1,4), (2,5), (3,6)]

9-element Array{Tuple{Int64,Int64},1}:
 (1, 2)
 (2, 3)
 (3, 4)
 (4, 5)
 (5, 6)
 (6, 1)
 (1, 4)
 (2, 5)
 (3, 6)

In [10]:
dg = dual_graph(edges)
einsum(Tuple(dg), Tuple(ϵ for i=1:length(dg)), ())

0-dimensional Array{Int64,0}:
12

# properties of s tensors

In [70]:
# define a delta tensor
function δ(n::Int)
    t = zeros(Int, fill(n, n) |> Tuple)
    for i=1:n
        t[fill(i, n)...] = 1
    end
    t
end

δ (generic function with 1 method)

In [37]:
using Test

In [38]:
@test einsum(((1,2,3),), (ϵ,), (1,3,2)) ≈ ϵ

Test Passed

In [39]:
@test einsum(((1,2,3),(1,2,3)), (ϵ, ϵ), ())[] ≈ 6

Test Passed

In [68]:
@test einsum(((1,2,3),(3,4,5)), (ϵ,ϵ), (1,2,4,5)) ≈
    einsum(((1,4),(2,5)), (I3,I3), (1,2,4,5)) +
    einsum(((1,5),(2,4)), (I3,I3), (1,2,4,5)) -
    2*einsum(((1,2,3),(3,4,5)), (δ(3),δ(3)), (1,2,4,5))

Test Passed

In [42]:
# idempotence
E = einsum(((1,2,3), (4,5,6)), (ϵ, ϵ), (1,2,3,4,5,6))/factorial(3)
@test E2 = einsum(((1,2,3,4,5,6), (4,5,6,7,8,9)), (E, E), (1,2,3,7,8,9)) ≈ E

Test Passed

In [44]:
@test einsum(((1,2,3), (2,3,4)), (ϵ, ϵ), (1,4)) ≈ 2*I

Test Passed

In [45]:
@test einsum(((1,2,3), (3,4,5), (5,6,1)), (ϵ, ϵ, ϵ), (2,4,6)) ≈ ϵ

Test Passed

In [51]:
@test einsum(((1,2,3), (3,4,5), (5,6,7), (7,8,1)), (ϵ, ϵ, ϵ, ϵ), (2,4,6,8)) ≈
    einsum(((2,8),(4,6)), (I3,I3), (2,4,6,8)) +
    einsum(((2,4),(8,6)), (I3,I3), (2,4,6,8))

Test Passed

In [53]:
@test einsum(((1,2,3), (3,4,5), (5,6,7), (7,8,9), (9,10,1)), (ϵ, ϵ, ϵ, ϵ, ϵ), (2,4,6,8,10)) ≈
    einsum(((2,4,6),(8,10)), (ϵ,I3), (2,4,6,8,10)) +
    einsum(((2,8,10),(4,6)), (ϵ,I3), (2,4,6,8,10)) +
    einsum(((6,8,10),(2,4)), (ϵ,I3), (2,4,6,8,10)) -
    einsum(((10,2,11),(11,4,12),(12,6,8)), (ϵ,ϵ,ϵ), (2,4,6,8,10))

Test Passed

## Quantum Computability

In [12]:
u2 = reshape(einsum(((1,2,3), (4,2,5)), (ϵ, ϵ), (1,4,3,5)), (9,9))

9×9 Array{Int64,2}:
 0  0  0  0  1  0  0  0  1
 0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  1  0  0
 0  1  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  1  0
 0  0  1  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0
 1  0  0  0  1  0  0  0  0

In [13]:
u2' ≈ u2

true

In [15]:
u1 = reshape(einsum(((1,2,3), (4,2,5)), (ϵ, δ(3)), (1,4,3,5)), (9,9))

9×9 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0

In [32]:
u1'*u1

9×9 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0

In [16]:
u2' ≈ u2

true

In [7]:
 h

9×9 Array{Int64,2}:
 2  0  0  0  1  0  0  0  1
 0  1  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0
 1  0  0  0  2  0  0  0  1
 0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  1  0
 1  0  0  0  1  0  0  0  2

In [26]:
U = (I+0.011im*u1)

9×9 Array{Complex{Float64},2}:
 1.0+0.0im  0.0+0.0im    0.0+0.0im    …  0.0+0.0im    0.0+0.0im    0.0+0.0im
 0.0+0.0im  1.0+0.0im    0.0+0.011im     0.0+0.0im    0.0+0.0im    0.0+0.0im
 0.0+0.0im  0.0+0.011im  1.0+0.0im       0.0+0.0im    0.0+0.0im    0.0+0.0im
 0.0+0.0im  0.0+0.0im    0.0+0.0im       0.0+0.0im    0.0+0.0im    0.0+0.0im
 0.0+0.0im  0.0+0.0im    0.0+0.0im       0.0+0.0im    0.0+0.0im    0.0+0.0im
 0.0+0.0im  0.0+0.0im    0.0+0.0im    …  0.0+0.0im    0.0+0.0im    0.0+0.0im
 0.0+0.0im  0.0+0.0im    0.0+0.0im       1.0+0.0im    0.0+0.011im  0.0+0.0im
 0.0+0.0im  0.0+0.0im    0.0+0.0im       0.0+0.011im  1.0+0.0im    0.0+0.0im
 0.0+0.0im  0.0+0.0im    0.0+0.0im       0.0+0.0im    0.0+0.0im    1.0+0.0im

In [25]:
U*U'

9×9 Array{Complex{Int64},2}:
 1+0im  0+0im  0+0im  0+0im  0+0im  0+0im  0+0im  0+0im  0+0im
 0+0im  2+0im  0+0im  0+0im  0+0im  0+0im  0+0im  0+0im  0+0im
 0+0im  0+0im  2+0im  0+0im  0+0im  0+0im  0+0im  0+0im  0+0im
 0+0im  0+0im  0+0im  2+0im  0+0im  0+0im  0+0im  0+0im  0+0im
 0+0im  0+0im  0+0im  0+0im  1+0im  0+0im  0+0im  0+0im  0+0im
 0+0im  0+0im  0+0im  0+0im  0+0im  2+0im  0+0im  0+0im  0+0im
 0+0im  0+0im  0+0im  0+0im  0+0im  0+0im  2+0im  0+0im  0+0im
 0+0im  0+0im  0+0im  0+0im  0+0im  0+0im  0+0im  2+0im  0+0im
 0+0im  0+0im  0+0im  0+0im  0+0im  0+0im  0+0im  0+0im  1+0im